In [10]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../..')))

In [11]:
from dotenv import load_dotenv
load_dotenv('../../.env')

True

In [12]:
from llama_cpp import Llama, LlamaGrammar
from llmail.connectors.imap import Inbox
import os

In [31]:
# Initialize Llama and Inbox
llm = Llama(model_path="../models/llama-2-7b-chat.ggmlv3.q2_K.bin")

llama.cpp: loading model from ../models/llama-2-7b-chat.ggmlv3.q2_K.bin


llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 10 (mostly Q2_K)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 2733.66 MB (+  256.00 MB per state)
llama_new_context_with_model: kv self size  =  256.00 MB
llama_new_context_with_model: compute buffer total size =   71.84 MB
AVX = 

In [41]:
username = os.getenv("MAIL_USER")
password = os.getenv("MAIL_PASS")
inbox = Inbox(username, password)

In [42]:
# Fetch the latest email
latest_emails = inbox.fetch_latest(10)
latest_email = latest_emails[0]

In [43]:
latest_email = latest_emails[1]
latest_email["subject"]

'New Course: Learn to finetune large language models'

In [44]:
# Combine email with a prompt
prompt = f"Q: Is this a recruitment email: 'YES' or 'NO'? Email: {latest_email['subject']} A:"

via the server: 
(run the openai compatible server with `python3 -m llama_cpp.server --model ./llmail/models/llama-2-13b-chat.ggmlv3.q4_0.bin`)

In [50]:
import requests
import json

# Prepare the data
data = {
    "messages": [{"role": "user", "content": prompt}]
}

# Make the POST request
response = requests.post(
    "http://localhost:8000/v1/chat/completions",
    headers={"Content-Type": "application/json"},
    data=json.dumps(data)
)

# Print the response
print(response.json())


{'id': 'chatcmpl-c52856aa-95bc-40b1-ab81-0ca35b5ca33b', 'object': 'chat.completion', 'created': 1692821319, 'model': './llmail/models/llama-2-13b-chat.ggmlv3.q4_0.bin', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': ' This is not a recruitment email. It is an announcement about a'}, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 44, 'completion_tokens': 16, 'total_tokens': 60}}


via python:

In [47]:
# Call Llama with the prompt
output = llm(prompt, max_tokens=32, stop=["Q:", "\n"], echo=True)

Llama.generate: prefix-match hit



llama_print_timings:        load time = 41719.22 ms
llama_print_timings:      sample time =   265.36 ms /    31 runs   (    8.56 ms per token,   116.82 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 53456.17 ms /    31 runs   ( 1724.39 ms per token,     0.58 tokens per second)
llama_print_timings:       total time = 54658.64 ms


In [48]:
output

{'id': 'cmpl-6f72ad06-1274-4411-84a9-8ccc79097a39',
 'object': 'text_completion',
 'created': 1692819151,
 'model': '../models/llama-2-7b-chat.ggmlv3.q2_K.bin',
 'choices': [{'text': "Q: Is this a recruitment email: 'YES' or 'NO'? Email: New Course: Learn to finetune large language models A: I would say 'NO' because the email is not specifically about recruiting people for a new course, but rather about a new course offering.",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 36, 'completion_tokens': 30, 'total_tokens': 66}}

set accepted answer format

In [24]:
grammar = r'''
root ::= answer
answer ::= "YES" | "NO" | "UNKNOWN"
'''
llama_grammar = LlamaGrammar.from_string(grammar)

root ::= answer 
answer ::= [Y] [E] [S] | [N] [O] | [U] [N] [K] [N] [O] [W] [N] 


from_string grammar:



In [29]:
output = llm(prompt, max_tokens=32, stop=["Q:", "\n"], echo=False, grammar=llama_grammar)

Llama.generate: prefix-match hit



llama_print_timings:        load time = 78967.50 ms
llama_print_timings:      sample time =    54.36 ms /     2 runs   (   27.18 ms per token,    36.79 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 11581.72 ms /     2 runs   ( 5790.86 ms per token,     0.17 tokens per second)
llama_print_timings:       total time = 11661.64 ms


In [26]:
output

{'id': 'cmpl-9b41daf0-33a5-4a4d-83a8-7a51e8630583',
 'object': 'text_completion',
 'created': 1692816055,
 'model': '../models/llama-2-13b-chat.ggmlv3.q4_0.bin',
 'choices': [{'text': 'NO',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 36, 'completion_tokens': 1, 'total_tokens': 37}}